# Molecular Learning
Sept. 11, 2017

## >Morning work
### Store molecule info in csv
Working on prac_csv.py

Attempting to store molecule information into a csv that I can merge with the data.csv that contains energy calculations.

The issue that arose with this was the molecule information got converted to a string when put into a csv causing issues when reading it out.

In [ ]:
import glob
import numpy as np
import pybel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
from matplotlib import style
import csv

ob = pybel.ob


def atomType(mol, atomIdx):
    # get the atomic type given an atom index
    return mol.OBMol.GetAtom(atomIdx).GetType()


row_list = []

# Read through all the files in the folder of this directory
for directory in glob.iglob("/Users/dakota/Documents/Research/conformers/*CHG_jobs/*"):
    name = "/".join(directory.split('/')[6:10])  # name of the entry

    for files in glob.iglob(directory + "/rmsd*.mol"):
        conf = files.split('/')[-1]  # conformer name/number

        try:
            mol = next(pybel.readfile('mol', files))
        except:
            pass

        bonds = []
        for bond in ob.OBMolBondIter(mol.OBMol):
            begin = atomType(mol, bond.GetBeginAtomIdx())
            end = atomType(mol, bond.GetEndAtomIdx())
            if (end < begin):
                # swap them for lexographic order
                begin, end = end, begin
            dict2 = {}
            bond_type = ("%s%s" % (begin[0], end[0]))
            bond_length = ("%8.4f" % (bond.GetLength()))
            dict2.update({'Bond Type': bond_type})
            dict2.update({'Bond Length': bond_length})
            bonds.append(dict2)

        dfb = pd.DataFrame(bonds, columns=['Bond Type', 'Bond Length'])
        # Encode the bond types into numerical representations for sklearn
        poss_types = {'CC': 0, 'CF': 1, 'CH': 2, 'CN': 3, 'CO': 4, 'CP': 5, 'CS': 6,
                      'HN': 7, 'HO': 8, 'OS': 9, 'OP': 10, 'NS': 11, 'NN': 12, 'NO': 13, 'BC': 14}
        dfb['Bond Type'] = dfb['Bond Type'].map(poss_types)
        # When iterating through molecueles with different number of bonds it will be 
        # .. important that the matrices are still the same length
        bond_difference = 500 - len(bonds)
        bond_extra = ([None] * bond_difference)
        dfb = pd.DataFrame.append(dfb, bond_extra)
        dfb = dfb.fillna(-99999)
        dfb.drop([0], inplace=True, axis=1)

        # store the descriptors for the molecules in a dictionary
        dict1 = {}
        dict1.update({'Molecule': dfb})
        dict1.update({'Name': name})
        dict1.update({'Conformer': conf})
        row_list.append(dict1)
        
        
# Make a dataframe of all the molecules, their descriptors and energies
df = pd.DataFrame(row_list, columns=['Name', 'Conformer', 'Molecule'])

df.to_csv('/Users/dakota/PycharmProjects/Research/MolecularLearning/molecular_learning/molecule.csv')



### Reading from csv
After this part was ran, the molecule information and energy information were merged into one csv file and organized to ensure the data properly lined up.

From here the there was an attempt to read the newly merged csv file.

In [4]:
df_molecules = pd.read_csv(
    '/Users/dakota/PycharmProjects/Research/MolecularLearning/molecular_learning/molecule.csv')

molecules = df_molecules['Molecule']
Energy = df_molecules['pm7']

X = np.asarray(list(molecules), dtype=np.float)
y = np.asarray(list(Energy), dtype=np.float)


ValueError: could not convert string to float: '     1         2\r0    0    1.3954\r1    0    1.3947\r2    0    1.3914\r3    6    1.7854\r4    0    1.3911\r5    0    1.3973\r6    2    1.0977\r7    0    1.4991\r8    0    1.3982\r9    0    1.4980\r10   0    1.5260\r11   2    1.1160\r12   2    1.1178\r13   3    1.5210\r14   3    1.5193\r15   7    1.0415\r16   7    1.0443\r17   2    1.1211\r18   2    1.1203\r19   9    1.4369\r20   9    1.4368\r21  11    1.7293\r22   2    1.0917\r23   2    1.1012\r24   2    1.1157\r25   2    1.1197\r26   7    1.0118\r27   7    1.0116'

### Not working and where to go from here
I kinda figured this way would not workout but was just trying to see if this was at all possible to do. I am going to go back to trying to merge the data.csv file into a pandas dataframe with the one created from reading through the molecule.

I may attempt this again, but from a different route. It might be beneficial in the future to somehow save all of the molecular information and then import it. Currently, the time requirement for my script (~20 secs for N=1700) stems mostly from reading through the molecules and organizing them, something I currently have to do everytime I want to run the ML on the molecules. The ML itself is not the bottleneck and finishes relatively quickly. It would be improve workflow if everytime I trained the ML that I did not have to read back through the molecules again.

## >Afternoon
### Merging w/ data.csv
This afternoon I will be working on merging data.csv that contains all of the energy values with molecule dataframe that I make in pandas. The main problem that needs fixing is ensuring that when I merge, the correct energy value lines up with the correct molecule. One main issue has been that the data.csv order goes pm7, opt, mmff which causes some issues when merging. It will merge on the correct number molecule, but it does not seem to put it with the correct optimization.

## >For tomorrow
Continue trying to make progress combining data.csv with the molecule dataframe